# Testing cobrapy deletion mechanism

In [1]:
import cobra
from cobra.flux_analysis import *

import pandas as pd


## importing 3 models in different variables 

In [2]:
model_a = cobra.io.read_sbml_model('AH_model.xml')

In [3]:
model_b =cobra.io.read_sbml_model('AH_model.xml')

In [4]:
model_c = cobra.io.read_sbml_model('AH_model.xml')

## Objective value of model_a

In [5]:
normal_a = model_a.optimize().objective_value
print(normal_a)

0.21420011654287383


## Perform all single reaction deletions on mode_b and checking its objective value

In [6]:
del_b = single_reaction_deletion(model_b,return_frame=True, processes= 1).reset_index()[['ids','growth']]

In [8]:
normal_b = model_b.optimize().objective_value
print(normal_b)

0.2142001042265253


## Hard coding single_reaction_deletion function on model_c and checking its objective value

In [17]:
def deletions(model):
    result = {}
    for reaction in model.reactions:
        with model as m: #create a context to prevent model from changing
            m.reactions.get_by_id(reaction.id).knock_out()
            sol = m.optimize()
            if sol.status == 'infeasible':#check infeasibility
                new = None
            else:
                new = sol.objective_value
            result[reaction.id] = new
    return pd.DataFrame(result.items(),columns=['ids', 'growth'])

In [10]:
del_c = deletions(model_c)

cobra\util\solver.py:416 UserWarning: solver status is 'infeasible'


In [12]:
normal_c = model_c.optimize().objective_value
print(normal_c)

0.21420010880736498


# Comparing the results

In [13]:
normal_a == normal_b

False

In [14]:
normal_a == normal_c

False

In [15]:
model_a == model_b

False

In [16]:
model_a == model_c

False

The solutions created aren't the same and these models are also different even though the function single_reaction_deletion isn't suposed to change the model as in the [source code](https://cobrapy.readthedocs.io/en/latest/_modules/cobra/flux_analysis/deletion.html) a context is used to avoid changes. I also tried to simulate a context myself in model_c and normal_c but the model still changed.

Finally, contexts should allow changes freely without altering the original model as stated in [cobrapy documentation](https://cobrapy.readthedocs.io/en/latest/getting_started.html#Making-changes-reversibly-using-models-as-contexts)